In [ ]:
import pickle
import warnings
import numpy as np
from tqdm.auto import tqdm
from scipy.io import loadmat

warnings.filterwarnings("ignore")


In [ ]:
TASKS = [
    "bert",
    "coref",
    "ner",
    "nli",
    "paraphrase",
    "qa",
    "sa",
    "srl",
    "ss",
    "sum",
    "wsd",
]
ROIS = [
    "language_lh",
    "language_rh",
    "vision_body",
    "vision_face",
    "vision_object",
    "vision_scene",
    "vision",
    "dmn",
    "task",
]
subjects = ["P01", "M02", "M04", "M07", "M15"]
layers_bert = [
    "block1",
    "block2",
    "block3",
    "block4",
    "block5",
    "block6",
    "block7",
    "block8",
    "block9",
    "block10",
    "block11",
    "block12",
    "fc",
]
layers_bart = ["fc"]


In [ ]:
def generate_indices(data):
    Taskindices = []
    for j in data["meta"][0][0][11][0][5]:
        for k in j[0]:
            Taskindices.append(int(k))
    DMNindices = []
    for j in data["meta"][0][0][11][0][6]:
        for k in j[0]:
            DMNindices.append(int(k))
    Visualindices = []
    Visualindices_body = []
    Visualindices_face = []
    Visualindices_object = []
    Visualindices_scene = []
    for j in data["meta"][0][0][11][0][9]:
        for k in j[0]:
            Visualindices_body.append(int(k))
    for j in data["meta"][0][0][11][0][10]:
        for k in j[0]:
            Visualindices_face.append(int(k))
    for j in data["meta"][0][0][11][0][11]:
        for k in j[0]:
            Visualindices_object.append(int(k))
    for j in data["meta"][0][0][11][0][12]:
        for k in j[0]:
            Visualindices_scene.append(int(k))

    for j in data["meta"][0][0][11][0][13]:
        for k in j[0]:
            Visualindices.append(int(k))
    Languageindices_lh = []
    Languageindices_rh = []
    for j in data["meta"][0][0][11][0][7]:
        for k in j[0]:
            Languageindices_lh.append(int(k))
    for j in data["meta"][0][0][11][0][8]:
        for k in j[0]:
            Languageindices_rh.append(int(k))
    return (
        Taskindices,
        DMNindices,
        Visualindices_body,
        Visualindices_face,
        Visualindices_object,
        Visualindices_scene,
        Visualindices,
        Languageindices_lh,
        Languageindices_rh,
    )


In [ ]:
def get_subject_data(subject):
    data_pic1 = loadmat("./pereira_dataset/" + subject + "/data_384sentences.mat")
    data_pic2 = loadmat("./pereira_dataset/" + subject + "/data_243sentences.mat")
    data_pic = loadmat("./pereira_dataset/" + subject + "/data_384sentences.mat")

    data_pic["examples_passagesentences"] = np.concatenate(
        (
            data_pic1["examples_passagesentences"],
            data_pic2["examples_passagesentences"],
        ),
        axis=0,
    )
    data_pic["meta"] = np.concatenate((data_pic1["meta"], data_pic2["meta"]), axis=0)
    (
        Taskindices,
        DMNindices,
        Visualindices_body,
        Visualindices_face,
        Visualindices_object,
        Visualindices_scene,
        Visualindices,
        Languageindices_lh,
        Languageindices_rh,
    ) = generate_indices(data_pic)

    roi_indices = {
        "language_lh": Languageindices_lh,
        "language_rh": Languageindices_rh,
        "vision_body": Visualindices_body,
        "vision_face": Visualindices_face,
        "vision_object": Visualindices_object,
        "vision_scene": Visualindices_scene,
        "vision": Visualindices,
        "dmn": DMNindices,
        "task": Taskindices,
    }

    fmri = {}
    for roi, indices in roi_indices.items():
        fmri[roi] = data_pic["examples_passagesentences"][0:, np.array(indices) - 1]

    return fmri


In [ ]:
def forward_pass(model, x):
    return model.predict(x)


def ensemble(outputs, subject, roi, typ="avg"):
    if typ == "avg":
        ret = np.mean(outputs, 0)
        return ret
    elif typ == "wt_avg":
        with open("./weights_wt_avg.pkl", "rb") as f:
            weights = pickle.load(f)
        ret = outputs.astype(np.float32)
        for i in range(ret.shape[0]):
            ret[i] = np.dot(ret[i], weights[subject][roi][i])
        ret = np.sum(ret, 0)
        return ret
    else:
        pass


In [ ]:
f_data = {}

for subject in tqdm(subjects):
    f_data[subject] = {}
    for roi in ROIS:
        outputs = []
        for task in TASKS:
            print(f"{subject} {roi} {task}")
            data = np.load(f"./features/pereira_{task}.npy", allow_pickle=True)
            feats = np.array(data[-1])
            with open(f"./models/{task}_{roi}_{subject}.model", "rb") as f:
                model = pickle.load(f)

            output = forward_pass(model, feats)
            outputs.append(output)
            del model, data

        outputs = np.array(outputs)
        f_output = ensemble(outputs, subject=subject, roi=roi, typ="wt_avg")
        f_data[subject][roi] = f_output


In [ ]:
with open("./ensemble_wt_avg.pkl", "wb") as f:
    pickle.dump(f_data, f)
